    Эти графики показывают общую ситуацию на сегодняшний момент по заказам,
     то есть они отвечают на вопрос - "Что происходит?"

In [2]:
import configparser

import altair as alt
import pandas as pd
import psycopg2

In [3]:
def format_cursor(cursor):
    desc = [desc[0] for desc in cursor.description]
    data_in_query = [{desc[j]: i[j] for j in range(len(desc))}
                      for i in cursor.fetchall()]

    return data_in_query

In [4]:
def data(data_in_query):
    df = pd.DataFrame(data_in_query)
    return df

In [5]:
config = configparser.ConfigParser()
config.read('TEST.ini', encoding="utf-8")

config = {i: config['postgresql'][i].replace('"', '')
           for i in config['postgresql']}

In [6]:
with psycopg2.connect(**config) as connect:
    with connect.cursor() as cursor_DAU:
        cursor_DAU.execute("""
        -- DAU (Daily Active Users) - Количество уникальных активных пользователей В ДЕНЬ
        -- WAU (Weekly Active Users) - Количество уникальных активных пользователей В НЕДЕЛЮ
        -- MAU (Monthly Active Users) - Количество уникальных активных пользователей В МЕСЯЦ

        -- DAU
        SELECT date_trunc('day', time) AS "date", COUNT(DISTINCT user_id) AS DAU FROM user_actions
        GROUP BY "date"
        """)
        df_DAU = data(format_cursor(cursor_DAU))

    with connect.cursor() as cursor_WAU:
        cursor_WAU.execute("""
        -- WAU (Поиск количества разделяя по понедельникам)
        SELECT date_trunc('week', time) "week",
        COUNT(DISTINCT user_id) WAU
        FROM user_actions
        GROUP BY "week"
        """)
        df_WAU = data(format_cursor(cursor_WAU))

    with connect.cursor() as cursor_MAU:
        cursor_MAU.execute("""
        -- MAU (Поиск количества разделяя по первым дням месяца)
        SELECT date_trunc('month', time) "month",
        COUNT(DISTINCT user_id) mau
        FROM user_actions
        GROUP BY "month"
        """)
        df_MAU = data(format_cursor(cursor_MAU))

    # Daily active couriers

    with connect.cursor() as cursor_DAC:
        cursor_DAC.execute("""
        SELECT COUNT(DISTINCT courier_id) as active_couriers,
        date_trunc('day', time) AS date
        FROM courier_actions
        GROUP BY 2
        """)
        df_DAC = data(format_cursor(cursor_DAC))

    # Daily active delivered orders

    with connect.cursor() as cursor_DADO:
        cursor_DADO.execute("""
        SELECT COUNT(order_id) as delivered_orders,
        date_trunc('day', time) AS date
        FROM courier_actions
        WHERE "action" LIKE 'del%'
        GROUP BY 2
        """)
        df_DADO = data(format_cursor(cursor_DADO))

print(df_DAU.dtypes, df_WAU.dtypes, df_MAU.dtypes, df_DAC.dtypes, df_DADO.dtypes, sep="\n \n")

date    datetime64[ns]
dau              int64
dtype: object
 
week    datetime64[ns]
wau              int64
dtype: object
 
month    datetime64[ns]
mau               int64
dtype: object
 
active_couriers             int64
date               datetime64[ns]
dtype: object
 
delivered_orders             int64
date                datetime64[ns]
dtype: object


In [7]:
# Daily active delivered orders
title = alt.TitleParams('Daily active delivered orders', anchor='middle', font='Verdana', fontSize=14, color='darkred')

DADO = alt.Chart(df_DADO, title=title).mark_line().encode(
    x='date',
    y='delivered_orders'
).properties(height=400, width=500)

DADO_text = DADO.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="delivered_orders:Q"
)

DADO = (DADO + DADO_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12,
)

In [8]:
# Daily active couriers
title = alt.TitleParams('Daily active couriers', anchor='middle', font='Verdana', fontSize=14, color='darkred')

DAC = alt.Chart(df_DAC, title=title).mark_line().encode(
    x='date',
    y='active_couriers'
).properties(height=400, width=500)

DAC_text = DAC.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="active_couriers:Q"
)

DAC = (DAC + DAC_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)

In [9]:

# DAU
DAU = alt.Chart(df_DAU).mark_line().encode(
    x='date',
    y='dau',
).properties(height=400,width=500)

DAU_text = DAU.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="dau:Q"
)

DAU = (DAU + DAU_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)

In [10]:
# WAU
WAU = alt.Chart(df_WAU).mark_line().encode(
    x='week',
    y='wau'
).properties(height=400,width=500)

WAU_text = WAU.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="wau:Q"
)

WAU = (WAU + WAU_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)

In [11]:
# MAU
MAU = alt.Chart(df_MAU).mark_line().encode(
    x='month',
    y='mau'
).properties(height=400,width=500)

MAU_text = MAU.mark_text(
    align='center',
    baseline='middle',
    dy=-13,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="mau:Q"
)

MAU = (MAU + MAU_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)

In [12]:
display(DADO, DAC, DAU, WAU, MAU)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [13]:
# Daily Active Cancel Orders
with connect.cursor() as cursor_DACO:
    cursor_DACO.execute("""
    SELECT COUNT(order_id) as delivered_orders,
    date_trunc('day', time) AS date
    FROM user_actions
    WHERE "action" LIKE 'can%'
    GROUP BY 2
    """)
    df_DACO = data(format_cursor(cursor_DACO))

print(df_DACO.dtypes)

delivered_orders             int64
date                datetime64[ns]
dtype: object


In [14]:
# Daily Active Cancel Orders
title = alt.TitleParams('Daily Active Cancel Orders', anchor='middle', font='Verdana', fontSize=14, color='darkred')

DACO = alt.Chart(df_DACO, title=title).mark_line().encode(
    x='date',
    y='delivered_orders'
).properties(height=400, width=500)

DACO_text = DACO.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="delivered_orders:Q"
)

DACO = (DACO + DACO_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)
display(DACO)

alt.LayerChart(...)

In [16]:
# cancel_rate_and_date
with connect.cursor() as cursor_date_orders_on_active_users:
    cursor_date_orders_on_active_users.execute("""
    SELECT *,
    ROUND(cancel_orders / create_orders::DECIMAL, 3)::FLOAT as cancel_rate,
    ROUND(create_orders / active_users::DECIMAL, 2)::FLOAT as orders_per_users
    FROM (
        SELECT COUNT(order_id) FILTER(WHERE "action" LIKE 'can%') as cancel_orders,
        COUNT(order_id) FILTER(WHERE "action" LIKE 'crea%') as create_orders,
        COUNT(DISTINCT user_id) as active_users,
        date_trunc('day', time) AS date
        FROM user_actions
        GROUP BY date) t1
    """)
    df_date_orders_on_active_users = data(format_cursor(cursor_date_orders_on_active_users))

print(df_date_orders_on_active_users.dtypes)

cancel_orders                int64
create_orders                int64
active_users                 int64
date                datetime64[ns]
cancel_rate                float64
orders_per_users           float64
dtype: object


In [17]:
title = alt.TitleParams('Cancel rate and Date', anchor='middle', font='Verdana', fontSize=14, color='darkred')

Cancel_rate_and_date = alt.Chart(df_date_orders_on_active_users, title=title).mark_line().encode(
    x='date',
    y='cancel_rate'
).properties(height=400, width=500)

Cancel_rate_and_date_text = Cancel_rate_and_date.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="cancel_rate:Q"
)

Cancel_rate_and_date = (Cancel_rate_and_date + Cancel_rate_and_date_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)
display(Cancel_rate_and_date)

alt.LayerChart(...)

In [18]:
title = alt.TitleParams('Orders Per Users', anchor='middle', font='Verdana', fontSize=14, color='darkred')

Cancel_rate_and_date = alt.Chart(df_date_orders_on_active_users, title=title).mark_line().encode(
    x='date',
    y='orders_per_users'
).properties(height=400, width=500)

Cancel_rate_and_date_text = Cancel_rate_and_date.mark_text(
    align='center',
    baseline='middle',
    dy=-20,  # Nudges text up so it doesn't appear on top of the bar
    color="darkred"
).encode(
    text="orders_per_users:Q"
)

Cancel_rate_and_date = (Cancel_rate_and_date + Cancel_rate_and_date_text).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    titleFontSize=12
)
display(Cancel_rate_and_date)

alt.LayerChart(...)

    Эти графики показывают общую ситуацию на сегодняшний момент по заказам,
     то есть они отвечают на вопрос - "Что происходит?"